In [24]:
import time
import keras
import math
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep

In [25]:
df = pd.read_csv('^NSEI_withoutnull.csv')
df = df.dropna(axis=0, how='any')
df = df.drop(['Date'], axis=1)
df['Future_Close'] = df[' Future_Close']
df = df.drop([' Future_Close'], axis=1)
df['Rat'] = df['Future_Close']/df['Close']
df['Rat'] = df['Rat']>1
df['Rat'] = df['Rat'].astype(float)
df = df.drop(['Future_Close'], axis=1)
df.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Open,High,Low,Close,Rat
0,5564.649902,5608.649902,5394.350098,5519.350098,1.0
1,5524.299805,5638.600098,5514.750000,5608.600098,1.0
2,5611.299805,5772.549805,5608.950195,5731.700195,0.0
3,5729.250000,5743.549805,5655.600098,5698.149902,0.0
4,5699.549805,5749.950195,5595.500000,5617.549805,1.0


In [26]:
def standard_scaler(X_train, X_test):
    train_samples, train_nx, train_ny = X_train.shape
    test_samples, test_nx, test_ny = X_test.shape
    
    X_train = X_train.reshape((train_samples, train_nx * train_ny))
    X_test = X_test.reshape((test_samples, test_nx * test_ny))
    
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    
    X_train = X_train.reshape((train_samples, train_nx, train_ny))
    X_test = X_test.reshape((test_samples, test_nx, test_ny))
    
    return X_train, X_test

In [27]:
def preprocess_data(stock, seq_len):
    amount_of_features = len(stock.columns)
    data = stock.as_matrix()
    
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index : index + sequence_length])
        
    result = np.array(result)
    row = round(0.9 * result.shape[0])
    train = result[: int(row), :]
    
    train, result = standard_scaler(train, result)
    
    X_train = train[:, : -1]
    y_train = train[:, -1][: ,-1]
    X_test = result[int(row) :, : -1]
    y_test = result[int(row) :, -1][ : ,-1]

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], amount_of_features))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], amount_of_features))  

    return [X_train, y_train, X_test, y_test]

In [28]:
def build_model(layers):
    model = Sequential()

    # By setting return_sequences to True we are able to stack another LSTM layer
    model.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(
        output_dim=layers[3]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss=keras.losses.binary_crossentropy, optimizer="rmsprop", metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

In [29]:
df = df.astype(float)

In [30]:
window = 5
X_train, y_train, X_test, y_test = preprocess_data(df[:: -1], window)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (2171, 5, 5)
y_train (2171,)
X_test (241, 5, 5)
y_test (241,)


In [31]:
model = build_model([X_train.shape[2], window, 100, 1])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=5, input_shape=(None, 5), return_sequences=True)`
  


Compilation Time :  0.03200793266296387


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`


In [32]:
model.fit(
    X_train,
    y_train,
    batch_size=768,
    nb_epoch=100,
    validation_split=0.2,
    verbose=2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1736 samples, validate on 435 samples
Epoch 1/100
0s - loss: 0.0782 - acc: 0.0000e+00 - val_loss: 0.1797 - val_acc: 0.0000e+00
Epoch 2/100
0s - loss: -7.3388e-03 - acc: 0.0000e+00 - val_loss: 0.1796 - val_acc: 0.0000e+00
Epoch 3/100
0s - loss: 0.0082 - acc: 0.0000e+00 - val_loss: 0.1764 - val_acc: 0.0000e+00
Epoch 4/100
0s - loss: 0.0523 - acc: 0.0000e+00 - val_loss: 0.3687 - val_acc: 0.0000e+00
Epoch 5/100
0s - loss: -3.2977e-02 - acc: 0.0000e+00 - val_loss: 0.3104 - val_acc: 0.0000e+00
Epoch 6/100
0s - loss: -1.3132e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 7/100
0s - loss: -8.0639e-02 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 8/100
0s - loss: -2.6914e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 9/100
0s - loss: -1.8800e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 10/100
0s - loss: -2.5242e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 11/100
0s - loss: -2.

0s - loss: -1.8544e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 91/100
0s - loss: -2.9447e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 92/100
0s - loss: -2.5822e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 93/100
0s - loss: -2.5203e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 94/100
0s - loss: -2.4046e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 95/100
0s - loss: -3.0081e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 96/100
0s - loss: -2.7729e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 97/100
0s - loss: -1.8739e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 98/100
0s - loss: -1.3569e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 99/100
0s - loss: -2.3290e-01 - acc: 0.0000e+00 - val_loss: 0.6218 - val_acc: 0.0000e+00
Epoch 100/100
0s - loss: -2.3802e-01 - acc: 0.0000e+00 - val_lo

In [33]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(np.abs(trainScore[0]))))

testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(np.abs(testScore[0]))))

Train Score: -0.04 MSE (0.21 RMSE)
Test Score: -1.09 MSE (1.04 RMSE)


In [34]:
import seaborn as sns

In [35]:
pred1 = model.predict(X_train)

In [36]:
print(pred1)

[[ 0.09349311]
 [ 0.08596593]
 [ 0.07702178]
 ..., 
 [-0.12954617]
 [-0.13265641]
 [-0.13223006]]
